In [1]:
import galois
import numpy as np
import numpy.linalg as linalg
import networkx as nx
from networkx.algorithms import isomorphism as iso
import itertools
import sys

np.set_printoptions(threshold=sys.maxsize)

# got this from stack exchange
def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = np.array2string(a, max_line_width=np.infty).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r' \\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

In [2]:
F = [[0,0,0,0],[1,0,1,1],[1,1,2,1],[1,2,3,3],[1,3,0,4],[0,0,0,0],[1,2,4,2]]
G1 = [(1,2),(3,4),(1,3,5),(2,4,5),(1,4,5)]
G2 = [(1,2),(3,4),(1,3,5),(2,4,5),(1,4,5),(2,3,5)]
G3 = [(1,2),(1,3),(2,3),(1,4,5),(2,4,5),(3,4,5)]
G4 = [(1,2,3),(1,2,4),(1,2,5),(1,3,4),(1,3,5),(1,4,5),(2,3,4),(2,3,5),(2,4,5)]

In [3]:
def getLFSRandArray(q, m, coeff, I, GF):
    k = int((q**m-1)/(q-1))  # number of columns in array
    LFSR = GF.Zeros(q**m-1)

    for i in range(0, len(I)):
        LFSR[i] = I[i]

    for i in range(m, q**m-1):
        LFSR[i] = -coeff[1]*LFSR[i-1]-coeff[2]*LFSR[i-2]-coeff[3]*LFSR[i-3]

    A = GF.Zeros((q**m, k))

    for i in range(0, q**m-1):
        A[i] = LFSR[0:k]
        LFSR = np.roll(LFSR, q**m-2)

    return ([LFSR, A])

In [4]:
def getColumnVectors(q, m, A):
    k = int((q**m-1)/(q-1))

    vectors = []

    for vector in A[0:k:, 0:m]:
        vectors.append(vector)

    return vectors

In [5]:
def findIndicies(vectors, M):
    """
    vectors - Set of vectors
    M - Matrix where we want to know the indicies of the columns in vectors
    """
    indicies = []

    for vector in M.T:
        indicies.append(np.flatnonzero((vector == vectors).all(1))[
                        0])
    return indicies


def getLinIndependent(vectors, q, m, GF):
    k = int((q**m-1)/(q-1))
    LinIndep = []
    LinDep = []
    LinIndepArray = np.zeros((k,k,k))

    for comb in itertools.combinations(vectors, m):
        M = GF.Zeros((3,3))

        for i in range(0, m):
           M[:, i] = comb[i]

        indices = findIndicies(vectors, M)

        # if indicies == [0, 2, 18]:
        #    print(M)
        #    print(linalg.matrix_rank(M))

        if linalg.matrix_rank(M) == m:
          LinIndep.append(indices)

          for perm in itertools.permutations(indices):
            LinIndepArray[perm[0]][perm[1]][perm[2]] = 1
        else:
           LinDep.append(indices)

    return [LinIndep, LinDep, LinIndepArray]

In [6]:
def getArrayAccessStructure(LinIndepArray, q):
    p = q**2 + q + 1
    AS = []

    for comb in itertools.combinations(np.arange(p),3):
        if comb[0] == 0 and LinIndepArray[comb] == 0:
            AS.append((comb[1],comb[2]))
        
        if comb[0] != 0 and LinIndepArray[comb] == 1:
            break_loop = False
            for comb2 in itertools.combinations(comb,2):
                if LinIndepArray[comb2 + (0,)] == 0:
                    break_loop = True

            if not break_loop:
                AS.append(comb)
                    
    return AS

In [7]:
def splitStructure(structure):
    size2 = []
    size3 = []

    for auth in structure:
        if len(auth) == 2:
            size2.append(auth)
        else:
            size3.append(auth)

    return [size2,size3]

In [8]:
def flatten(lst):
    l = []
    for elt in lst:
        t = type(elt)
        if t is tuple or t is list:
            for elt2 in flatten(elt):
                l.append(elt2)
        else:
            l.append(elt)
    return sorted(set(l))

In [9]:
def isAuth(P,structure):
    for minAuth in structure:
        if set(minAuth).issubset(set(P)):
            return True
    return False

In [10]:
# Function that returns all unauthorized sets of size 2 and 3
def getUnauth(structure):
    n = max(flatten(structure))
    unAuth = []

    for comb in itertools.combinations(flatten(structure),2):
        if not isAuth(comb,structure):
            unAuth.append(comb)

    for comb in itertools.combinations(flatten(structure),3):
        if not isAuth(comb,structure):
            unAuth.append(comb)

    return unAuth

In [11]:
def buildGraphsAS(arrayStructure, accessStructure):
    ArrayUnauth = getUnauth(arrayStructure)
    AccessUnauth = getUnauth(accessStructure)

    arrayGraph = nx.Graph()
    accessGraph = nx.Graph()


    for auth in arrayStructure:
        if len(auth) == 2:
            arrayGraph.add_node(auth, color='green')
        else:
            arrayGraph.add_node(auth, color='yellow')

        for node in auth:
            arrayGraph.add_node(node, color='red')
            arrayGraph.add_edge(node, auth)

    for unauth in ArrayUnauth:
        if len(unauth) == 2:
            arrayGraph.add_node(unauth, color='blue')
        else:
            arrayGraph.add_node(unauth, color='purple')

        for node in unauth:
            arrayGraph.add_node(node, color='red')
            arrayGraph.add_edge(node, unauth)

    for auth in accessStructure:
        if len(auth) == 2:
            accessGraph.add_node(auth, color='green')
        else:
            accessGraph.add_node(auth, color='yellow')

        for node in auth:
            accessGraph.add_node(node, color='red')
            accessGraph.add_edge(node, auth)

    for unauth in AccessUnauth:
        if len(unauth) == 2:
            accessGraph.add_node(unauth, color='blue')
        else:
            accessGraph.add_node(unauth, color='purple')

        for node in unauth:
            accessGraph.add_node(node, color='red')
            accessGraph.add_edge(node, unauth)


    return [accessGraph, arrayGraph]

In [12]:
def buildGraphsLD(p, q, m, StrLinIndep, StrLinDep, ALinIndep, ALinDep):
    """
    p - number of participants
    q - size of finite field
    m - degree of polynomial
    StrLinIndep - Array representing linear independence in access structure
    StrLinDep - Array representing the linear dependence in access structure
    ALinIndep - Array represeting linear independence in A
    ALinDep - Array representing linear dependence in A
    """
    k = int((q**m-1)/(q-1))
    accessGraph = nx.Graph()
    arrayGraph = nx.Graph()

    accessGraph.add_node(0, color='red')
    arrayGraph.add_node(0, color='red')

    for i in range(1, p+1):
        accessGraph.add_node(i, color='purple')

    for i in range(1, k):
        arrayGraph.add_node(i, color='purple')

    for indep in tuple(map(tuple,StrLinIndep)):
        accessGraph.add_node(indep, color='blue')
        for node in indep:
            accessGraph.add_edge(node, indep)

    for dep in tuple(map(tuple,StrLinDep)):
        accessGraph.add_node(dep, color='yellow')
        for node in dep:
            accessGraph.add_edge(node, dep)

    for indep in tuple(map(tuple, ALinIndep)):
        arrayGraph.add_node(indep, color='blue')
        for node in indep:
            arrayGraph.add_edge(node, indep)

    for dep in tuple(map(tuple, ALinDep)):
        arrayGraph.add_node(dep, color='yellow')
        for node in dep:
            arrayGraph.add_edge(node, dep)

    return [accessGraph, arrayGraph]

In [13]:
def computeAccessStructure(AccessStr, p):
    parties = list(range(1, p+1))
    AccessStrLinIndep = []
    AccessStrLinDep = []
    for comb in itertools.combinations(parties, 2):
        if comb in AccessStr:
          AccessStrLinDep.append((0,) + comb)
        else:
           AccessStrLinIndep.append((0,) + comb)

    for comb in itertools.combinations(parties,3):
        AccessStrLinDep.append(comb)

    for set in AccessStr:
        if len(set) == 2:
            for per in parties:
              if per not in set:
                new_tup = tuple(sorted((per,) + set))
                for comb in itertools.combinations(new_tup,2):
                  if comb not in AccessStr and new_tup in AccessStrLinDep:
                    AccessStrLinIndep.append(new_tup)
                    AccessStrLinDep.remove(new_tup)
                    break

        if len(set) == 3:
            AccessStrLinIndep.append(set)
            AccessStrLinDep.remove(set)

    return [AccessStrLinIndep, AccessStrLinDep]

In [14]:
def setupAS(q, GF, m, I, coeff, AccessStr, p):
    [LFSR, A] = getLFSRandArray(q,m,coeff,I,GF)

    vectors = getColumnVectors(q,m,A)

    [LinIndep, LinDep, LinIndepArray] = getLinIndependent(vectors, q, m, GF)
    arrayStructure = getArrayAccessStructure(LinIndepArray,q)

    [AccessStrLinIndep, AccessStrLinDep] = computeAccessStructure(AccessStr, p)

    [AccessStrGraph, ArrayGraph] = buildGraphsAS(arrayStructure,AccessStr)
    
    return [LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, AccessStrGraph, ArrayGraph]

In [15]:
def setupLD(q, GF, m, I, coeff, AccessStr, p):
    [LFSR, A] = getLFSRandArray(q,m,coeff,I,GF)

    vectors = getColumnVectors(q,m,A)

    [LinIndep, LinDep, LinIndepArray] = getLinIndependent(vectors, q, m, GF)

    [AccessStrLinIndep, AccessStrLinDep] = computeAccessStructure(AccessStr, p)

    [AccessStrGraph, ArrayGraph] = buildGraphsLD(p,q,3,AccessStrLinIndep,AccessStrLinDep,LinIndep,LinDep)
    
    return [LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, AccessStrGraph, ArrayGraph]

In [16]:
def colors_match(n1_attrib,n2_attrib):
    '''returns False if either does not have a color or if the colors do not match'''
    try:
        return n1_attrib['color']==n2_attrib['color']
    except KeyError:
        return False

In [17]:
def computeIso(arrayGraph,accessGraph):
    Iso = iso.GraphMatcher(arrayGraph,accessGraph,node_match=colors_match)

    if Iso.subgraph_is_isomorphic():
        mapping = Iso.mapping
        isom = {}

        for v in mapping:
            if (type(v) != tuple and type(mapping[v]) != tuple) and type(v) != str:
                isom[mapping[v]] = v

        return isom
    
    return None

In [18]:
def init(q):
    GF = galois.GF(q)
    m = 3  # degree of polynomial

    I = [GF(1), GF(0), GF(0)]  # vector of size m
    coeff = [GF(F[q-1][0]),GF(F[q-1][1]),GF(F[q-1][2]),GF(F[q-1][3])] # coefficients of primitive polynomial (... + c2x^2 + c1x + c0)

    return [GF,m,I,coeff]

In [19]:
q = 7
p = 6
GF = galois.GF(q)

[GF,m,I,coeff] = init(q)

AS = [(1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (1, 6),
  (2, 3),
  (2, 4),
  (2, 5),
  (2, 6),
  (3, 4),
  (3, 5),
  (3, 6),
  (4, 5),
  (4, 6),
  (5, 6)]

[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraph, arrayGraph] = setupAS(q, GF, m, I, coeff, AS, p)

In [20]:
%%time
isom = computeIso(arrayGraph,accessGraph)

CPU times: total: 172 ms
Wall time: 185 ms


In [21]:
print(isom)

{1: 1, 2: 4, 3: 52, 4: 37, 5: 14, 6: 12}


In [30]:
q = 4
p = 6
GF = galois.GF(q)

[GF,m,I,coeff] = init(q)

AS = [(1, 2, 3),
  (1, 2, 4),
  (1, 2, 5),
  (1, 2, 6),
  (1, 3, 4),
  (1, 3, 5),
  (1, 3, 6),
  (1, 4, 5),
  (1, 4, 6),
  (1, 5, 6)]
[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraph, arrayGraph] = setupAS(q, GF, m, I, coeff, AS, p)

In [31]:
%%time
isom = computeIso(arrayGraph,accessGraph)

CPU times: total: 1h 19min 50s
Wall time: 1h 24min 40s


In [27]:
print(isom)

{1: 1, 2: 4, 3: 14, 4: 16, 5: 9, 6: 20}


In [ ]:
q = 3
p = 5
GF = galois.GF(q)

[GF,m,I,coeff] = init(q)

AS = G2

[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraph, arrayGraph] = setupAS(q, GF, m, I, coeff, AS, p)

In [21]:
%%time
isom = computeIso(arrayGraph,accessGraph)

CPU times: total: 3.41 s
Wall time: 3.65 s


In [22]:
print(isom)

None


In [20]:
q = 4
p = 5
GF = galois.GF(q)

[GF,m,I,coeff] = init(q)

AS = G4

[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraph, arrayGraph] = setupLD(q, GF, m, I, coeff, AS, p)

In [21]:
%%time

isom = computeIso(arrayGraph,accessGraph)

CPU times: total: 3h 16min 38s
Wall time: 3h 21min 25s


In [23]:
print(isom)

None


In [53]:
AS = G1
q = 7
p = 5
GF = galois.GF(q)

[GF,m,I,coeff] = init(q)

[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraphAS, arrayGraphAS] = setupAS(q, GF, m, I, coeff, AS, p)
[LinIndep, LinIndepArray, AccessStrLinIndep, AccessStrLinDep, accessGraphLD, arrayGraphLD] = setupLD(q, GF, m, I, coeff, AS, p)

print(len(arrayGraphAS.nodes), len(arrayGraphAS.edges))
print(len(arrayGraphLD.nodes), len(arrayGraphLD.edges))

20804 60704
29317 87780
